# Contextual Bandits with TF-agents







## Learning Objectives

* Learn to load a dataset in BigQuery and connect to it using TensorFlow I/O
* Learn how to transform a classification dataset into contextual bandit problem

*Contextual Bandit (CB)* is a machine learning framework in which a *agent* selects actions (also called *arms*) in order to maximize rewards in the long term. At each round, the agent receives some information about the current state (also called the *context*) and uses this information to select an action. As a consequence of this choice, it receives a *reward*. 

On the one hand, contextual bandit is one of the simplest instance of a *reinforcement learning problem* where a single state (or context) is provided to the agent and the play or *episode* stops after the first action has been chosen and the reward gotten. This setting appears in a number of useful problems in the industry, one of the best known being that of ad placements on a website: The different ads to publish on a webpage are the different actions, the context is given by a user features, and the reward is 1 is the user clicks on the published ad and 0 otherwise.

On the other hand, contextual bandit is a natural generalization of a classification problem in supervized learning. Namely, consider a data set of points $(x, y)$ where the $x$'s are the features and the $y$'s are the labels in $k$ possible classes. We can setup an associated contextual bandit problem as follows: The CB agent at each time step is given the context $x$. From that information, it needs to select from $k$ possible actions which are the $k$ possible classes. If the agent chooses the correct class for feature $x$, then the reward is $1$, and zero otherwise. The general goal of maximising the long-term cumulative reward for the CB agent is equivalent to that of minimizing the training loss in supervized learning. Contextual bandit is more general than classification though, since in many useful CB settings we actually know the reward only for the  actions we have taken. 

In this lab, we will learn how to solve a contextual bandit problem derived from a classification dataset with *Q-learning* and the associated *neural epsilon-greedy strategy* using a powerful reinforcement learning library written in tensorflow: [TensorFlow Agents](https://www.tensorflow.org/agents). 

**Acknowledgement:** This lab is based on a tutorial originally written by Anant Nawalgaria and Alex Erfurt. We thank them for making their original material available to us.

## Setup

Let us intall [TensorFlow Agents](https://www.tensorflow.org/agents) if it is not already installed and import the necessary libraries:

In [ ]:
pip freeze | grep tf_agents || pip install -q tf_agents

In [1]:
import functools
import os
import time

import pandas as pd
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import

# from tensorflow.python.framework.dtypes import int64
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_probability import distributions as tfd
from tf_agents.bandits.agents.neural_epsilon_greedy_agent import (
    NeuralEpsilonGreedyAgent,
)
from tf_agents.bandits.environments import environment_utilities as env_util
from tf_agents.bandits.environments.classification_environment import (
    ClassificationBanditEnvironment,
)
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks.q_network import QNetwork
from tf_agents.replay_buffers.tf_uniform_replay_buffer import (
    TFUniformReplayBuffer,
)

In [2]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

os.environ["PROJECT_ID"] = PROJECT_ID

## Loading the dataset into BigQuery

In this lab, we are going to use a classification dataset and turn it into a contextual bandit problem. 

Our dataset will be the [UCI Machine Learning Repository]( https://archive.ics.uci.edu/ml/datasets/covertype), which associates various cartographic features of a given area with different labels representing different types of forests covering the areas. 

The original features are as follows (the last column being the label):

In [3]:
pd.read_csv("../../tfx_pipelines/data/dataset.csv").head(2)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3142,183,9,648,101,757,223,247,157,1871,Commanche,C7757,1
1,2156,18,28,0,0,1207,187,170,107,960,Cache,C6102,3


At each time step, our CB agent will be given a context $x$ representing an area cartographic features (`Elevation`, `Aspect`, `Slope`, etc.). Then it will have to choose among one of 7 possible forest cover types as defined by the last column (`Cover_type`), and represented by the integer from 0 to 6.

For convenience, we have pre-precessed the categorical features `Wilderness_Area` and `Soil_Type` into their one-hot-encoded versions. So the dataset we will use will have more columns (55 exactly) than the original covertype dataset. We will name the columns corresponding to the 54 features from `X0` to `X54`, while the last column `Y` represents the label.

The next cell defines our dataset column names and types and displays a few examples:

In [4]:
DATASET_SOURCE = "../data/covertype.csv"

df = pd.read_csv(DATASET_SOURCE, header=None)

LABEL_NAME = "Y"
FEATURE_PREFIX = "X"
N_SAMPLES, N_COLUMNS = df.shape

COLUMN_NAMES = [f"{FEATURE_PREFIX}{i}" for i in range(N_COLUMNS - 1)] + [
    LABEL_NAME
]

COLUMN_TYPES = [tf.int64] * N_COLUMNS

df.columns = COLUMN_NAMES
df.head(2)

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X45,X46,X47,X48,X49,X50,X51,X52,X53,Y
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5


Let us now load this dataset into `BigQuery` into the table named

```bash
PROJECT_ID.DATASET_ID.TABLE_ID
```

where `DATASET_ID` and `TABLE_ID` are defined in the next cell among other variable like the `DATASET_SCHEMA`:

In [5]:
DATASET_LOCATION = "US"
DATASET_SCHEMA = ",".join([f"{name}:INTEGER" for name in COLUMN_NAMES])
DATASET_ID = "contextual_bandit"
TABLE_ID = "covertype"

os.environ["DATASET_LOCATION"] = DATASET_LOCATION
os.environ["DATASET_SOURCE"] = DATASET_SOURCE
os.environ["DATASET_SCHEMA"] = DATASET_SCHEMA
os.environ["DATASET_ID"] = DATASET_ID
os.environ["TABLE_ID"] = TABLE_ID

### Exercise

In the cell below run the [bq command line](https://cloud.google.com/bigquery/docs/bq-command-line-tool) to create the dataset and populate the table from `DATASET_SOURCE` using the variable defined above:

In [ ]:
%%bash

bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID

bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
--source_format=CSV \
--replace \
$TABLE_ID \
$DATASET_SOURCE \
$DATASET_SCHEMA

## Connecting to BigQuery

We will now create a `tf.data.Dataset` connected to the data table we created in our `BigQuery`.

For that purpose, we will use [Tensorflow_io](https://github.com/tensorflow/io/tree/v0.15.0/tensorflow_io/bigquery), which offers a connector `BigQueryClient` to  stream data directly out of `BigQuery`:

```python
from tensorflow_io.bigquery import BigQueryClient
```

The first step is to create a`BigQuery` client and then a read session from it:

In [6]:
bq_client = BigQueryClient()

bq_session = bq_client.read_session(
    f"projects/{PROJECT_ID}",
    PROJECT_ID,
    TABLE_ID,
    DATASET_ID,
    COLUMN_NAMES,
    COLUMN_TYPES,
)

2022-02-18 19:18:11.492101: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-02-18 19:18:11.492709: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2022-02-18 19:18:11.612444: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


From our `bq_session` we can create a `tf.data.Dataset` using the `parallel_read_rows` method, which will read our BigQuery rows in parallel: 

In [7]:
tf_dataset = bq_session.parallel_read_rows(
    block_length=N_SAMPLES,
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

At this point the examples are stored in our `tf_dataset` as `OrderedDict` with keys the column names and values the corresponding row values:

In [8]:
for example in tf_dataset.take(1):
    print(example)

2022-02-18 19:18:13.083264: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-18 19:18:13.097697: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-02-18 19:18:13.097748: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.


OrderedDict([('X0', <tf.Tensor: shape=(), dtype=int64, numpy=3244>), ('X1', <tf.Tensor: shape=(), dtype=int64, numpy=54>), ('X10', <tf.Tensor: shape=(), dtype=int64, numpy=1>), ('X11', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X12', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X13', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X14', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X15', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X16', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X17', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X18', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X19', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X2', <tf.Tensor: shape=(), dtype=int64, numpy=40>), ('X20', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X21', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X22', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X23', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X24', <tf.Tensor: shape=(), dtype=int64, numpy=0>), ('X25', <tf.T

### Exercise

Configure the `tf_dataset` we instanciated so that
1. the examples are stored as couples $(x, y)$ where $x$ is the feature vector with 54 components and $y$ is the label (**Hint:** Use `.map`)
1. it loops over the dataset infefinitively (**Hint:** Use `.repeat`)
1. it shuffles the dataset (Use `buffer_size=400000`)

In [9]:
# TODO
def features_and_labels(features):
    label = features.pop(LABEL_NAME)
    return (
        tf.cast(tf.stack(tf.nest.flatten(features), axis=0), tf.float32),
        tf.cast(label - 1, tf.int32),
    )


tf_dataset = (
    tf_dataset.map(features_and_labels).repeat().shuffle(buffer_size=400000)
)

Verify that now the dataset has the correct form:

In [10]:
for example in tf_dataset.take(1):
    print(example)

2022-02-18 19:18:33.778903: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-02-18 19:18:33.778959: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-02-18 19:18:43.691481: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 158577 of 400000
2022-02-18 19:18:53.691623: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 322921 of 400000


(<tf.Tensor: shape=(54,), dtype=float32, numpy=
array([2.526e+03, 7.900e+01, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.400e+01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.080e+02,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.800e+01, 0.000e+00,
       0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 5.100e+02, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 2.380e+02, 2.130e+02, 1.030e+02, 3.000e+02],
      dtype=float32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


2022-02-18 19:18:58.052730: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


## Initializing and configuring the CB environment

In Tensorflow Agents, there are special classes that provide the contexts and the rewards to the agent. These classe are generally called [environments](https://www.tensorflow.org/agents/tutorials/2_environments_tutorial). 

The environment defines the type of actions, states/contexts/observations, and rewards allowed in the problem through its `observation_spec`, `action_spec`, and `reward_spec` methods. In TensorFlow agent the environment are specific to a given problem but the agents are generic. This means that if you want to solve your own RL problem with TensorFlow agent, you'll likely have to write the environment that represents your problem and defines the type of actions and states allowed. Then you'll be able to use any of the generic agents in the TensorFlow agents library. The agent will adapt to your problem using `observation_spec`, `action_spec`, and `reward_spec`.

In this section, we will instanciate the environment to solve our "covertype bandit" problem.

In the TF-Agents bandits library, there is a special environment class named `ClassificationBanditEnvironment` that  turns any multiclass labeled dataset into a bandit environment. The contexts (or observations) will be the features in the dataset, the actions are the label classes.

In general, the rewards can be sampled from a probability distribution depending on the actual and the guessed labels. 
In our case, the rewards will be deterministic: The agent will receive 1 if it chooses the right class, and 0 otherwise. 

The next cell creates this reward structure using Tensorflow Probability:

In [11]:
covertype_reward_distribution = tfd.Independent(
    tfd.Deterministic(tf.eye(7)), reinterpreted_batch_ndims=2
)

If we sample from it, we obtain a $7\times 7$ identity matrix storing the rewards obtained by the agent if the agent selects class $i$ (corresponding to row $i$) when the actual class is $j$ (corresponding to column $j$):

In [12]:
covertype_reward_distribution.sample()

<tf.Tensor: shape=(7, 7), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]], dtype=float32)>

### Exercise

Instanciate the [ClassificationBanditEnvironment](https://www.tensorflow.org/agents/api_docs/python/tf_agents/bandits/environments/classification_environment/ClassificationBanditEnvironment) and invoke its `reward_spec`, `observation_spec`, and `action_spec` methods to make sure they correspond to the covertype bandit problem. 
(Note that the `ClassificationBanditEnvironment` can process many actions in parallel so it takes a `batch_size` argument to define the number of actions it will process simultaneously. Let set that batch size to 1 for now.)


In [13]:
# TODO: Define the environment
BATCH_SIZE = 1

environment = ClassificationBanditEnvironment(
    tf_dataset, covertype_reward_distribution, BATCH_SIZE
)

In [14]:
# TODO: Inspect the reward_spec
environment.reward_spec()

TensorSpec(shape=(), dtype=tf.float32, name='reward')

In [15]:
# TODO: Inspect the observation_spec
environment.observation_spec()

TensorSpec(shape=(54,), dtype=tf.float32, name=None)

In [16]:
# TODO: Inspect the action_spec
environment.action_spec()

BoundedTensorSpec(shape=(), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(6, dtype=int32))

### Exercise

Use the `reset` method on the `environment` you have just instanciated to obtain the first step.
Inspect this step `observation` attribute to see which cartographic features the environment has given you to guess the covertype. Then in a next cell guess a possible covertype from 0 to 6, and pass it to the environment using the `step` method, which returns the next step. Inspect the next step `reward` attribute to see if you guessed correctly.

In [17]:
# TODO
first_step = environment.reset()
first_step.observation

2022-02-18 19:19:01.372292: E tensorflow/core/framework/dataset.cc:552] Unimplemented: Cannot compute input sources for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-02-18 19:19:01.372367: E tensorflow/core/framework/dataset.cc:556] Unimplemented: Cannot merge options for dataset of type IO>BigQueryDataset, because the dataset does not implement `InputDatasets`.
2022-02-18 19:19:11.226051: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 114622 of 400000
2022-02-18 19:19:21.223402: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 223084 of 400000
2022-02-18 19:19:31.223374: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 366617 of 400000
2022-02-18 19:19:33.211270: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


<tf.Tensor: shape=(1, 54), dtype=float32, numpy=
array([[ 2.727e+03,  1.300e+01,  1.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  3.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  3.000e+01,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -1.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         1.320e+03,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         2.160e+02,  2.330e+02,  1.540e+02,  1.991e+03]], dtype=float32)>

In [18]:
# TODO
next_step = environment.step(3)
next_step.reward

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

The next step retunred by the environment also contains the new context:

In [19]:
next_step.observation

<tf.Tensor: shape=(1, 54), dtype=float32, numpy=
array([[3.083e+03, 1.080e+02, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.200e+01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.550e+02,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 3.700e+01, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 3.276e+03, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 2.400e+02, 2.260e+02, 1.150e+02, 8.340e+02]],
      dtype=float32)>

## Initializing the Agent

In the Tensorflow Agents, the classes that implement algorithms to solve the contextual bandit problem posed by the environment are called *agents*.   

In this lab, we will will use the [NeuralEpsilonGreedyAgent](https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870), which implements the *neural epsilon greedy algorithm*.

This algorithm predicts the reward for each possible action given a context as input using a neural network, called a [Q-Network](https://www.tensorflow.org/agents/api_docs/python/tf_agents/networks/q_network/QNetwork), since it outputs the $Q$-values $Q(x, a)$, that is, the predicted rewards for each of the actions $a$ given the an input context $x$.

The action is then chosen to be one with maximal $Q$-value with probability $1-\epsilon$ or a random action with probability $\epsilon$. This randomness allows the algorithm to explore states that could be promising even though the neural network estimate their reward poorly. 


### Exercise

Instanciate a [Q-Network](https://www.tensorflow.org/agents/api_docs/python/tf_agents/networks/q_network/QNetwork) so that it takes as 
* input tensor an observation tensor as defined by the environment 
* output tensor an action tensor as defined by the environment


Configure the structure of the layers as you wish.

In [31]:
# TODO
LAYERS = (300, 200, 100, 100, 50, 50)

network = QNetwork(
    input_tensor_spec=environment.observation_spec(),
    action_spec=environment.action_spec(),
    fc_layer_params=LAYERS,k
)

### Exercise

Generate a step from the environment using the `step`method, and feed its observation attribute to the $Q$-network.
Verify that the tensor of $Q$-values you are getting is of the right shape (you should get a vector with 7 components containing the predicted reward for each of the covertype classes):

In [34]:
# TODO
action = 2
step = environment.step(action)
q_values = network(step.observation)
q_values

(<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
 array([[ 78.622574, 310.8624  , -93.516   ,  54.339664, 124.87689 ,
         246.5174  ,  86.49705 ]], dtype=float32)>,
 ())

### Exercise

Now that we have our `QNetwork` to estimate our action rewards, in the next cell, you will instanciate the agent from the [NeuralEpsilonGreedyAgent](https://www.tensorflow.org/agents/api_docs/python/tf_agents/bandits/agents/neural_epsilon_greedy_agent/NeuralEpsilonGreedyAgent) class. You will need to retrieve the `time_step_spec` and the `action_spec` from the environment. The `reward_network` will be the `QNetwork` you instanciated previously. You can take `Adam` as optimizer with a `LEARNING_RATE` of your choice. 
You will also set the propensity of your agent to explorate rather than exploit the action with highest predicted reward through the value of `EPSILON`. Both the `LEARNING_RATE` and the `EPSILON` greediness are hyper-paramaters that can affect the training of th agent very much.

In [36]:
EPSILON = 0.01
LEARNING_RATE = 0.002

# TODO
agent = NeuralEpsilonGreedyAgent(
    time_step_spec=environment.time_step_spec(),
    action_spec=environment.action_spec(),
    reward_network=network,
    optimizer=tf.compat.v1.train.AdamOptimizer(LEARNING_RATE),
    epsilon=EPSILON,
)


The agent has a `policy` attribute containing the strategy that the agent will use when confronted to a given context. The `agent.policy` has an`action` method that takes in a `TimeStep` generated by the environment and containing the context. It then issues a `PolicyStep` containing the action chosen by the policy given this contex:

In [56]:
policy_step = agent.policy.action(step)
policy_step.action

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

Under the hood, the agent policy uses the `QNetwork` to predict the values of the different actions (or possible covertype classes in our case). However at this stage the `QNetwork` has not been trained and its weights are random. This means that the predicted classes are meaningless.

To remediate that, the agent has a `train` method that takes an experience, that is, a triples of a state (or context) $x$, the action taken (or predicted class) $y$, and the obtained reward $r$. It then updates the parameters $\theta$ of the `QNetwork` $Q_\theta$ with a gradient update so that the predicted reward $Q_\theta(x, y)$ used by the agent to guess $y$ in context $x$ increases if $y$ is the correct class (i.e. reward $1$) and decreases if $y$ has been guessed wrongly.

Now training one experience at a time may lead to an unstable training, and the algorithm may have difficulty to converge. To stabilize the training one collects a number of these experience in a *experience replay buffer* in a first stage, and then sample batches of these experiences to apply the gradient update to the `QNetwork` parameters in a second stage, as we do in supervized learning. 

## Collecting experiences in an experience replay buffer

Reinforcement learning algorithms use replay buffers to store trajectories of experience when executing a policy in an environment. During training, replay buffers are queried for a subset of the trajectories (either a sequential subset or a sample) to "replay" the agent's experience. Sampling from the replay buffer facilitate data re-use and breaks harmful co-relation between sequential data in RL, although in contextual bandits this isn't absolutely required but still helpful.

Tensorflow Agents defines a number of [replay buffer classes](https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial) all sharing a common interface to store and access the experience data. In this lab, we will use [TFUniformReplayBuffer](https://www.tensorflow.org/agents/api_docs/python/tf_agents/replay_buffers/TFUniformReplayBuffer) which samples uniformly experience trajectories. 

## Exercise


Initialize a [TFUniformReplayBuffer](https://www.tensorflow.org/agents/api_docs/python/tf_agents/replay_buffers/TFUniformReplayBuffer)  using for `data_spec` the `trajectory_spec` stored in the agent's policy. Then use a `batch_size` of 128, which will be the size of the batches sampled from the replay buffer for each gradient descent update. The`max_length` argument indicates the maximum number of steps we allow to be stored in the replay buffer from a single episode. Since for CB the eposide size is always 1, you'll set this argument to that. 


In [60]:
# TODO

BATCH_SIZE = 128
STEPS_PER_LOOP = 2

replay_buffer = TFUniformReplayBuffer(
    data_spec=agent.policy.trajectory_spec,
    batch_size=BATCH_SIZE,
    max_length=STEPS_PER_LOOP,
)

Now we have a Replay buffer, but we also need something to fill it with. Often a common practice is to have 
the agent interact with and collect experiences from the environment, without actually learning from it (that is without updating the parameters of the `QNetwork`) in a first step. 

This data-collection loop can be carried out using a [DynamicStepDriver](https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers/dynamic_step_driver/DynamicStepDriver), which will 
1. feed the `TimeStep` generated by the environment and containing the context data for the new step and the reward for the previous step to the agent,
1. collect the action generated by the agent policy from the environment contex, and then 
1. feed that action back to the environment.  

All that in a loop. 

The data encountered by the driver at each step is saved in a `NamedTuple` called [Trajectory](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/Trajectory) and broadcast to a set of observers such as replay buffers. 
This trajectory includes the observation from the environment, the action recommended by the policy, the reward obtained, the type of the current and the next step, etc. 

In order for the driver to fill the replay buffer with data it needs acess to the [add_batch method](https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial#data_collection) of the replay buffer.


## Exercise

Instanciate [DynamicStepDriver](https://www.tensorflow.org/agents/api_docs/python/tf_agents/drivers/dynamic_step_driver/DynamicStepDriver) using the environment we used so far. Note that the agent has two different policies:
* `agent.policy` which is *exploitation policy* policy that outputs the action with maximal predicted reward. This is the policy that should be deployed in production environment and used when trained.
* `agent.collect_policy` which is the *exploration policy* that outputs the maximal reward action only $1 - \epsilon$ of the time, allowing for exploring a wider range of actions. This is the policy that is the most beneficial to use to collect training data, and the one that the `DynmicStepDriver` needs to use. 

In [68]:
observer = [replay_buffer.add_batch]

# TODO
driver = DynamicStepDriver(
    env=environment,
    policy=agent.collect_policy,
    num_steps=100,  # STEPS_PER_LOOP * environment.batch_size,
    observers=observer,
)

In [ ]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=BATCH_SIZE,
    num_steps=STEPS_PER_LOOP,
    single_deterministic_pass=True,
)

experience, unused_info = next(iter(dataset))

train_loss = agent.train(experience).loss

buf.clear()

## Train the agent


For experimentations, the batchize was set to ... now for real training, let us redefine all the ...

In [20]:
TIMESTAMP = time.strftime("%Y%m%d_%H%M%S")
ROOT_DIR = f"./contextual_bandit_checkpoints/{TIMESTAMP}"

TRAINING_LOOPS = 10
STEPS_PER_LOOP = 2
AGENT_ALPHA = 10.0


AGENT_CHECKPOINT_NAME = "agent"
STEP_CHECKPOINT_NAME = "step"
CHECKPOINT_FILE_PREFIX = "ckpt"

Here we provide you with a helper function in order to save your agent, the metrics and its lighter policy seperately, while training the model. We make all the aspects into trackable objects and then use checkpoint to save as well warm restart a previous training. For more information on checkpoints and policy savers ( which will be used in the training loop below) refer [here](https://www.tensorflow.org/agents/tutorials/10_checkpointer_policysaver_tutorial)

In [ ]:
def restore_and_get_checkpoint_manager(root_dir, agent, metrics, step_metric):
    """Restores from `root_dir` and returns a function that writes checkpoints."""
    trackable_objects = {metric.name: metric for metric in metrics}
    trackable_objects[AGENT_CHECKPOINT_NAME] = agent
    trackable_objects[STEP_CHECKPOINT_NAME] = step_metric
    checkpoint = tf.train.Checkpoint(**trackable_objects)
    checkpoint_manager = tf.train.CheckpointManager(
        checkpoint=checkpoint, directory=root_dir, max_to_keep=5
    )
    latest = checkpoint_manager.latest_checkpoint

    if latest is not None:
        print("Restoring checkpoint from %s.", latest)
        checkpoint.restore(latest)
        print("Successfully restored to step %s.", step_metric.result())
    else:
        print(
            "Did not find a pre-existing checkpoint. " "Starting from scratch."
        )
    return checkpoint_manager

In [ ]:
checkpoint_manager = restore_and_get_checkpoint_manager(
    ROOT_DIR, agent, metrics, step_metric
)
summary_writer = tf.summary.create_file_writer(ROOT_DIR)
summary_writer.set_as_default()

Now we have all the components ready to start training the model. Here is the process for Training the model
1. We first use the DynamicStepdriver instance to collect experience (trajectories) from the environment and fill up the replay buffer.
2. We then extract all the stored experience from the replay buffer by specfiying the `batch size` and `num_steps` we initialized the driver with. We extract it as `tf.dataset.Dataset` instance.
3. We then iterate on the `tf.dataset.Dataset` and the first sample we draw actually has all the data `batch_size * num_time_steps`
4. The agent then trains on the acquired experience
5. The replay buffer is cleared to make space for new data
6. Log the metrics and store them on disk
7. Save the Agent ( via checkpoints) as well as the policy

## Metrics

Just like you have metrics like accuracy/recall in supervised learning, in bandits we use the [regret](https://www.tensorflow.org/agents/tutorials/bandits_tutorial#regret_metric) metric per episode. To calculate the regret, we need to know what the highest possible expected reward is in every time step. For that, we define the `optimal_reward_fn`.

Another similar metric is the number of times a suboptimal action was chosen. That requires the definition if the `optimal_action_fn`.

In [ ]:
optimal_reward_fn = functools.partial(
    env_util.compute_optimal_reward_with_classification_environment,
    environment=environment,
)

optimal_action_fn = functools.partial(
    env_util.compute_optimal_action_with_classification_environment,
    environment=environment,
)

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)

suboptimal_arms_metric = tf_bandit_metrics.SuboptimalArmsMetric(
    optimal_action_fn
)

step_metric = tf_metrics.EnvironmentSteps()

metrics = [
    # equivalent to number of steps in bandits problem
    tf_metrics.NumberOfEpisodes(),
    # measures regret
    regret_metric,
    # number of times the suboptimal arms are pulled
    suboptimal_arms_metric,
    # the average return
    tf_metrics.AverageReturnMetric(batch_size=environment.batch_size),
]

In [ ]:
replay_observer = [replay_buffer.add_batch, step_metric] + metrics

driver = DynamicStepDriver(
    env=environment,
    policy=agent.collect_policy,
    num_steps=STEPS_PER_LOOP * environment.batch_size,
    observers=replay_observer,
)

In [ ]:
# solution
import warnings

warnings.filterwarnings("ignore")
TRAINING_LOOPS = 150

for _ in range(TRAINING_LOOPS):
    driver.run()
    batch_size = driver.env.batch_size

    dataset = buf.as_dataset(
        sample_batch_size=BATCH_SIZE,
        num_steps=STEPS_PER_LOOP,
        single_deterministic_pass=True,
    )

    experience, unused_info = next(iter(dataset))

    train_loss = agent.train(experience).loss

    buf.clear()

    metric_utils.log_metrics(metrics)
    # for m in metrics:
    # print(m.name, ": ", m.result())
    for metric in metrics:
        metric.tf_summaries(train_step=step_metric.result())
    checkpoint_manager.save()

    # saver.save(os.path.join(ROOT_DIR, "./", 'policy_%d' % step_metric.result()))

Now that our model is trained, what if we want to determine which action to take given a new "context": for that we will iterate on our dataset to get the next item,
    make a timestep out of it by wrapping the results using `ts.TimeStep`. It expects `step_type`, `reward`, `discount`, and `observation` as input: since we are performing prediction you can fill 
        in dummy values for the first 3: only the observation/context is relevant. Read about how it works [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/time_step/TimeStep), and perform the task below
        
       

In [ ]:
feature, label = iter(tf_dataset).next()

In [ ]:
step = ts.TimeStep(
    tf.constant(ts.StepType.FIRST, dtype=tf.int32, shape=[1], name="step_type"),
    tf.constant(0.0, dtype=tf.float32, shape=[1], name="reward"),
    tf.constant(1.0, dtype=tf.float32, shape=[1], name="discount"),
    tf.constant(feature, dtype=tf.float32, shape=[1, 54], name="observation"),
)

agent.policy.action(step).action.numpy()

One final task : let us upload the tensoboard logs, to get an overview of the performance of our model. We will upload our logs to `tensorboard.dev` and for that you need to 
copy the following command in terminal and execute it from there, it will give you a link from which you need to copy/paste the authentication code, and once that is done, you will receive the 
url of your model evaluation, hosted on a public [tensorboard.dev](https://tensorboard.dev/) instance

In [ ]:
!tensorboard dev upload --logdir /home/jupyter/tmp/quick_test/v7/ --name "(optional) My latest experiment" --description "(optional) Agent trained"